In [41]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd 
from openff.recharge.esp.storage import MoleculeESPStore
from openff.units import Unit, unit
from openff.toolkit.topology import Molecule, FrozenMolecule
import numpy as np

In [2]:
esp_store = MoleculeESPStore("./esp-store.sqlite", cache_size=20000000)

In [34]:
table = pq.read_table('testing.parquet')
pandas_parquet = table.to_pandas()
pandas_parquet

,smiles,conformation,dipole,mbis-charges
0,[C:1]1([H:6])([H:7])[C:2]([H:8])([H:9])[C:3]1(...,"[-1.4508960799973198, 2.2138234699959103, 0.57...","[-0.7245480110433249, 0.39570986852366735, 0.4...","[-0.2745272536510941, -0.2745272536510941, 0.0..."
1,[C:1]([Br:2])([Br:3])([H:4])[H:5],"[-0.17883947999966965, 0.2143222999996041, 0.1...","[-0.3746714841307508, 0.44901139881855556, 0.4...","[-0.10192293609198132, -0.129839397361932, -0...."
2,[c:1]1([H:11])[c:2]([H:12])[c:3]([H:13])[c:4]2...,"[3.816001649992951, -0.43151211999920286, 1.24...","[0.4103150346159055, -0.05157396125295576, 0.0...","[-0.17305135342221262, -0.1047265937364485, -0..."
3,[C:1]1([H:10])=[C:2]([Br:9])[C:3]2=[C:7]([S:6]...,"[-1.7542141599967596, -2.764151429994894, 2.46...","[0.13901726959062444, -1.5253246683835755, 0.4...","[-0.02154374296728488, -0.021672326701921946, ..."
4,[C:1]1([H:13])=[C:5]([Br:6])[N:4]([H:14])[C:3]...,"[1.6749715699969059, -4.577603939991544, 0.892...","[-1.6464586576679636, 4.278692035598283, -0.99...","[-0.40241659410844, 0.33456349482330905, -0.06..."
...,...,...,...,...
6799,[C:1]1([H:13])=[N:5][O:4][N:3]=[C:2]1[S:6][C:7...,"[-4.692321609991332, -2.831535599994769, -5.28...","[5.19928247168666, -2.848307882353083, 2.07801...","[0.018750378577532586, 0.21846580830943366, -0..."
6800,[C:1]1([H:13])=[N:5][S:4][C:3]2=[C:2]1[C:9]([H...,"[-2.8758186299946873, -2.1326943799960603, -3....","[1.089039350778691, 2.4849390366413893, 4.4280...","[0.19253340704793356, -0.11989556449834911, -0..."
6801,[C:1]1([H:13])=[N:5][S:4][N:3]=[C:2]1[C@:6]12[...,"[-5.011166799990743, -3.9707130999926656, -1.4...","[1.4993679527814834, 0.8681353750157257, 0.239...","[0.04265233646163001, 0.2889285700181526, -0.5..."
6802,[C:1]1([H:10])=[C:2]2[C:3](=[C:4]([H:11])[S:5]...,"[2.0726634299961715, -6.316279109988332, -0.65...","[0.7947236276989997, 0.12701131824780987, 0.47...","[-0.3721182542703619, 0.2678683831916664, 0.26..."


In [61]:
pandas_parquet['smiles']

0       [C:1]1([H:6])([H:7])[C:2]([H:8])([H:9])[C:3]1(...
1                       [C:1]([Br:2])([Br:3])([H:4])[H:5]
2       [c:1]1([H:11])[c:2]([H:12])[c:3]([H:13])[c:4]2...
3       [C:1]1([H:10])=[C:2]([Br:9])[C:3]2=[C:7]([S:6]...
4       [C:1]1([H:13])=[C:5]([Br:6])[N:4]([H:14])[C:3]...
                              ...                        
6799    [C:1]1([H:13])=[N:5][O:4][N:3]=[C:2]1[S:6][C:7...
6800    [C:1]1([H:13])=[N:5][S:4][C:3]2=[C:2]1[C:9]([H...
6801    [C:1]1([H:13])=[N:5][S:4][N:3]=[C:2]1[C@:6]12[...
6802    [C:1]1([H:10])=[C:2]2[C:3](=[C:4]([H:11])[S:5]...
6803    [C:1]12=[N:5][O:4][N:3]=[C:2]1[N:6]([H:13])[C:...
Name: smiles, Length: 6804, dtype: object

In [37]:
pandas_parquet['canon_smiles'] = pandas_parquet.apply(lambda row: canonical_smiles(row['smiles'], row['conformation']), axis=1)

/scratch/ipykernel_22495/1928538295.py:4: AtomMappingWarning: Warning! Fully mapped SMILES pattern passed to `from_smiles`. The atom map is stored as a property in `Molecule._properties`, but these indices are NOT used to determine atom ordering. To use these indices for atom ordering, use `Molecule.from_mapped_smiles`.
  smiles_mol = Molecule.from_smiles(smiles, allow_undefined_stereo=True)


In [39]:
# Check for duplicates and raise warnings if necessary
if pandas_parquet['canon_smiles'].duplicated().any():
    print("Warning: There are duplicate canonical SMILES entries. These will be ignored.")

In [59]:
pandas_parquet

,smiles,conformation,dipole,mbis-charges,canon_smiles,inch
0,[C:1]1([H:6])([H:7])[C:2]([H:8])([H:9])[C:3]1(...,"[-1.4508960799973198, 2.2138234699959103, 0.57...","[-0.7245480110433249, 0.39570986852366735, 0.4...","[-0.2745272536510941, -0.2745272536510941, 0.0...",BrC1(Br)CC1,InChI=1/C3H4Br2/c4-3(5)1-2-3/h1-2H2
1,[C:1]([Br:2])([Br:3])([H:4])[H:5],"[-0.17883947999966965, 0.2143222999996041, 0.1...","[-0.3746714841307508, 0.44901139881855556, 0.4...","[-0.10192293609198132, -0.129839397361932, -0....",BrCBr,InChI=1/CH2Br2/c2-1-3/h1H2
2,[c:1]1([H:11])[c:2]([H:12])[c:3]([H:13])[c:4]2...,"[3.816001649992951, -0.43151211999920286, 1.24...","[0.4103150346159055, -0.05157396125295576, 0.0...","[-0.17305135342221262, -0.1047265937364485, -0...",Brc1[nH]nc2ccccc12,InChI=1/C7H5BrN2/c8-7-5-3-1-2-4-6(5)9-10-7/h1-...
3,[C:1]1([H:10])=[C:2]([Br:9])[C:3]2=[C:7]([S:6]...,"[-1.7542141599967596, -2.764151429994894, 2.46...","[0.13901726959062444, -1.5253246683835755, 0.4...","[-0.02154374296728488, -0.021672326701921946, ...",Brc1c[nH]c2sncc12,"InChI=1/C5H3BrN2S/c6-4-2-7-5-3(4)1-8-9-5/h1-2,7H"
4,[C:1]1([H:13])=[C:5]([Br:6])[N:4]([H:14])[C:3]...,"[1.6749715699969059, -4.577603939991544, 0.892...","[-1.6464586576679636, 4.278692035598283, -0.99...","[-0.40241659410844, 0.33456349482330905, -0.06...",Brc1cc2c([nH]1)[C@@H]1C[NH2+]C[C@H]1O2,InChI=1/C8H9BrN2O/c9-7-1-5-8(11-7)4-2-10-3-6(4...
...,...,...,...,...,...,...
6799,[C:1]1([H:13])=[N:5][O:4][N:3]=[C:2]1[S:6][C:7...,"[-4.692321609991332, -2.831535599994769, -5.28...","[5.19928247168666, -2.848307882353083, 2.07801...","[0.018750378577532586, 0.21846580830943366, -0...",c1nonc1SC1CC[NH2+]CC1,InChI=1/C7H11N3OS/c1-3-8-4-2-6(1)12-7-5-9-11-1...
6800,[C:1]1([H:13])=[N:5][S:4][C:3]2=[C:2]1[C:9]([H...,"[-2.8758186299946873, -2.1326943799960603, -3....","[1.089039350778691, 2.4849390366413893, 4.4280...","[0.19253340704793356, -0.11989556449834911, -0...",c1nsc2c1C[NH2+]CC21C[NH2+]C1,InChI=1/C8H11N3S/c1-6-2-11-12-7(6)8(3-9-1)4-10...
6801,[C:1]1([H:13])=[N:5][S:4][N:3]=[C:2]1[C@:6]12[...,"[-5.011166799990743, -3.9707130999926656, -1.4...","[1.4993679527814834, 0.8681353750157257, 0.239...","[0.04265233646163001, 0.2889285700181526, -0.5...",c1nsnc1[C@@]12C[N@H+]3C[C@@H]1[C@@H]2C3,InChI=1/C8H9N3S/c1-7(10-12-9-1)8-4-11-2-5(8)6(...
6802,[C:1]1([H:10])=[C:2]2[C:3](=[C:4]([H:11])[S:5]...,"[2.0726634299961715, -6.316279109988332, -0.65...","[0.7947236276989997, 0.12701131824780987, 0.47...","[-0.3721182542703619, 0.2678683831916664, 0.26...",c1scc2c1OCCO2,"InChI=1/C6H6O2S/c1-2-8-6-4-9-3-5(6)7-1/h3-4H,1..."


In [4]:
smiles = '[C:1]([C:2]([C:3]([H:14])([H:15])[H:16])([C:4]1=[C:8]([H:18])[N:7]([C:9]([H:19])([H:20])[H:21])[N:6]=[C:5]1[H:17])[N+:10]([H:22])([H:23])[H:24])([H:11])([H:12])[H:13]'

pandas_parquet.loc[pandas_parquet['smiles'] == smiles]

,smiles,conformation,dipole,mbis-charges


In [50]:
is_in = Molecule.from_smiles(pandas_parquet.iloc[0]['smiles']).to_smiles(explicit_hydrogens=False)
is_in

/scratch/ipykernel_22495/512611018.py:1: AtomMappingWarning: Warning! Fully mapped SMILES pattern passed to `from_smiles`. The atom map is stored as a property in `Molecule._properties`, but these indices are NOT used to determine atom ordering. To use these indices for atom ordering, use `Molecule.from_mapped_smiles`.
  is_in = Molecule.from_smiles(pandas_parquet.iloc[0]['smiles']).to_smiles(explicit_hydrogens=False)


'BrC1(Br)CC1'

In [48]:
is_in_conf = pandas_parquet.iloc[0]['conformation'].reshape(-1,3)*unit.bohr
is_in_conf

Magnitude,[[-1.4508960799973198 2.2138234699959103 0.5706609099989458] [-2.0452842999962217 -0.3044418099994376 1.7481529799967708] [0.09266095999982883 -0.05060609999990652 -0.061398599999886574] [-0.26156917999951684 -1.51034239999721 -3.3621127799937893] [3.47554292999358 -0.2449497899995475 1.2306163999977267] [-0.5807500199989272 3.6384376299932786 1.7258761099968118] [-2.692882899995025 2.9232670999945998 -0.869949559998393] [-1.5788390699970833 -0.5902610999989095 3.7031308799931595] [-3.691008289993182 -1.3055272999975884 1.1073574499979546]]
Units,bohr


In [49]:
frozen_molecule = FrozenMolecule.from_smiles(is_in)
frozen_molecule._add_conformer(is_in_conf)
FrozenMolecule.are_isomorphic(mol1=frozen_molecule, mol2=frozen_molecule, return_atom_map=True)

(True, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8})

In [52]:
is_in = Molecule.from_smiles(pandas_parquet.iloc[0]['smiles'])
is_in_conf = pandas_parquet.iloc[0]['conformation'].reshape(-1,3)*unit.bohr
is_in.add_conformer(is_in_conf)
FrozenMolecule.are_isomorphic(mol1=is_in, mol2=is_in, return_atom_map=True)

/scratch/ipykernel_22495/2235373035.py:1: AtomMappingWarning: Warning! Fully mapped SMILES pattern passed to `from_smiles`. The atom map is stored as a property in `Molecule._properties`, but these indices are NOT used to determine atom ordering. To use these indices for atom ordering, use `Molecule.from_mapped_smiles`.
  is_in = Molecule.from_smiles(pandas_parquet.iloc[0]['smiles'])


(True, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8})

In [67]:
def canonical_smiles(smiles: str, conformer: list) -> str:
    """return canonical smiles with same toolkit"""
    conformer = np.array(conformer).reshape(-1, 3)
    smiles_mol = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
    smiles_mol.add_conformer(coordinates=conformer * unit.angstrom)
    return smiles_mol.to_smiles(explicit_hydrogens=False)

def canonical(smiles: str) -> str:
    """return canonical smiles with same toolkit"""
    smiles_mol = Molecule.from_mapped_smiles(smiles, allow_undefined_stereo=True)
    return smiles_mol.to_smiles(explicit_hydrogens=False)

def openff_mol(smiles: str, conformer: unit.Quantity) -> Molecule:
    """make an openff_mol"""
    conformer = np.array(conformer).reshape(-1, 3)
    smiles_mol = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
    smiles_mol.add_conformer(coordinates=conformer * unit.angstrom)
    return smiles_mol
    
def inchi(smiles: str, conformer: unit.Quantity) -> str:
    """return an inchi from a smiles"""
    openffmol =  openff_mol(smiles, conformer)
    return openffmol.to_inchi(fixed_hydrogens=True)

def remap(mol1: Molecule, mol2: Molecule) -> Molecule:
    """remap mol1 to mol2 if they are isomorphic"""
    
    if (isomap := FrozenMolecule.are_isomorphic(mol1=mol1, mol2=mol2, return_atom_map=True)[0]):
        map_dictionary = isomap[1]

    mol1.remap(mapping_dict=map_dictionary)
    
    return mol1

def isclose(conf1: list, conf2: list) -> bool:
    """test if two lists are close to one another"""
    np_conf1, np_conf2 = np.array(conf1), np.array(conf2)
    return np.isclose(np_conf1, np_conf2)


In [68]:
pandas_parquet['inchi'] = pandas_parquet.apply(
    lambda row: inchi(row['smiles'], row['conformation']), axis = 1
)


/scratch/ipykernel_22495/3220913835.py:16: AtomMappingWarning: Warning! Fully mapped SMILES pattern passed to `from_smiles`. The atom map is stored as a property in `Molecule._properties`, but these indices are NOT used to determine atom ordering. To use these indices for atom ordering, use `Molecule.from_mapped_smiles`.
  smiles_mol = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
[15:48:59] WARNING: Proton(s) added/removed; Ambiguous stereo: center(s)

[15:48:59] WARNING: Proton(s) added/removed

[15:48:59] WARNING: Proton(s) added/removed

[15:48:59] WARNING: Proton(s) added/removed

[15:48:59] WARNING: Proton(s) added/removed

[15:48:59] WARNING: Ambiguous stereo: center(s)

[15:48:59] WARNING: Ambiguous stereo: center(s)

[15:48:59] WARNING: Ambiguous stereo: center(s)

[15:48:59] WARNING: Proton(s) added/removed; Ambiguous stereo: center(s)

[15:48:59] WARNING: Proton(s) added/removed; Ambiguous stereo: center(s)

[15:48:59] WARNING: Proton(s) added/removed; Ambiguous 

In [69]:
pandas_parquet   

,smiles,conformation,dipole,mbis-charges,canon_smiles,inch,inchi
0,[C:1]1([H:6])([H:7])[C:2]([H:8])([H:9])[C:3]1(...,"[-1.4508960799973198, 2.2138234699959103, 0.57...","[-0.7245480110433249, 0.39570986852366735, 0.4...","[-0.2745272536510941, -0.2745272536510941, 0.0...",BrC1(Br)CC1,InChI=1/C3H4Br2/c4-3(5)1-2-3/h1-2H2,InChI=1/C3H4Br2/c4-3(5)1-2-3/h1-2H2
1,[C:1]([Br:2])([Br:3])([H:4])[H:5],"[-0.17883947999966965, 0.2143222999996041, 0.1...","[-0.3746714841307508, 0.44901139881855556, 0.4...","[-0.10192293609198132, -0.129839397361932, -0....",BrCBr,InChI=1/CH2Br2/c2-1-3/h1H2,InChI=1/CH2Br2/c2-1-3/h1H2
2,[c:1]1([H:11])[c:2]([H:12])[c:3]([H:13])[c:4]2...,"[3.816001649992951, -0.43151211999920286, 1.24...","[0.4103150346159055, -0.05157396125295576, 0.0...","[-0.17305135342221262, -0.1047265937364485, -0...",Brc1[nH]nc2ccccc12,InChI=1/C7H5BrN2/c8-7-5-3-1-2-4-6(5)9-10-7/h1-...,InChI=1/C7H5BrN2/c8-7-5-3-1-2-4-6(5)9-10-7/h1-...
3,[C:1]1([H:10])=[C:2]([Br:9])[C:3]2=[C:7]([S:6]...,"[-1.7542141599967596, -2.764151429994894, 2.46...","[0.13901726959062444, -1.5253246683835755, 0.4...","[-0.02154374296728488, -0.021672326701921946, ...",Brc1c[nH]c2sncc12,"InChI=1/C5H3BrN2S/c6-4-2-7-5-3(4)1-8-9-5/h1-2,7H","InChI=1/C5H3BrN2S/c6-4-2-7-5-3(4)1-8-9-5/h1-2,7H"
4,[C:1]1([H:13])=[C:5]([Br:6])[N:4]([H:14])[C:3]...,"[1.6749715699969059, -4.577603939991544, 0.892...","[-1.6464586576679636, 4.278692035598283, -0.99...","[-0.40241659410844, 0.33456349482330905, -0.06...",Brc1cc2c([nH]1)[C@@H]1C[NH2+]C[C@H]1O2,InChI=1/C8H9BrN2O/c9-7-1-5-8(11-7)4-2-10-3-6(4...,InChI=1/C8H9BrN2O/c9-7-1-5-8(11-7)4-2-10-3-6(4...
...,...,...,...,...,...,...,...
6799,[C:1]1([H:13])=[N:5][O:4][N:3]=[C:2]1[S:6][C:7...,"[-4.692321609991332, -2.831535599994769, -5.28...","[5.19928247168666, -2.848307882353083, 2.07801...","[0.018750378577532586, 0.21846580830943366, -0...",c1nonc1SC1CC[NH2+]CC1,InChI=1/C7H11N3OS/c1-3-8-4-2-6(1)12-7-5-9-11-1...,InChI=1/C7H11N3OS/c1-3-8-4-2-6(1)12-7-5-9-11-1...
6800,[C:1]1([H:13])=[N:5][S:4][C:3]2=[C:2]1[C:9]([H...,"[-2.8758186299946873, -2.1326943799960603, -3....","[1.089039350778691, 2.4849390366413893, 4.4280...","[0.19253340704793356, -0.11989556449834911, -0...",c1nsc2c1C[NH2+]CC21C[NH2+]C1,InChI=1/C8H11N3S/c1-6-2-11-12-7(6)8(3-9-1)4-10...,InChI=1/C8H11N3S/c1-6-2-11-12-7(6)8(3-9-1)4-10...
6801,[C:1]1([H:13])=[N:5][S:4][N:3]=[C:2]1[C@:6]12[...,"[-5.011166799990743, -3.9707130999926656, -1.4...","[1.4993679527814834, 0.8681353750157257, 0.239...","[0.04265233646163001, 0.2889285700181526, -0.5...",c1nsnc1[C@@]12C[N@H+]3C[C@@H]1[C@@H]2C3,InChI=1/C8H9N3S/c1-7(10-12-9-1)8-4-11-2-5(8)6(...,InChI=1/C8H9N3S/c1-7(10-12-9-1)8-4-11-2-5(8)6(...
6802,[C:1]1([H:10])=[C:2]2[C:3](=[C:4]([H:11])[S:5]...,"[2.0726634299961715, -6.316279109988332, -0.65...","[0.7947236276989997, 0.12701131824780987, 0.47...","[-0.3721182542703619, 0.2678683831916664, 0.26...",c1scc2c1OCCO2,"InChI=1/C6H6O2S/c1-2-8-6-4-9-3-5(6)7-1/h3-4H,1...","InChI=1/C6H6O2S/c1-2-8-6-4-9-3-5(6)7-1/h3-4H,1..."


In [57]:
pandas_parquet

,smiles,conformation,dipole,mbis-charges,canon_smiles,inch
0,[C:1]1([H:6])([H:7])[C:2]([H:8])([H:9])[C:3]1(...,"[-1.4508960799973198, 2.2138234699959103, 0.57...","[-0.7245480110433249, 0.39570986852366735, 0.4...","[-0.2745272536510941, -0.2745272536510941, 0.0...",BrC1(Br)CC1,InChI=1/C3H4Br2/c4-3(5)1-2-3/h1-2H2
1,[C:1]([Br:2])([Br:3])([H:4])[H:5],"[-0.17883947999966965, 0.2143222999996041, 0.1...","[-0.3746714841307508, 0.44901139881855556, 0.4...","[-0.10192293609198132, -0.129839397361932, -0....",BrCBr,InChI=1/CH2Br2/c2-1-3/h1H2
2,[c:1]1([H:11])[c:2]([H:12])[c:3]([H:13])[c:4]2...,"[3.816001649992951, -0.43151211999920286, 1.24...","[0.4103150346159055, -0.05157396125295576, 0.0...","[-0.17305135342221262, -0.1047265937364485, -0...",Brc1[nH]nc2ccccc12,InChI=1/C7H5BrN2/c8-7-5-3-1-2-4-6(5)9-10-7/h1-...
3,[C:1]1([H:10])=[C:2]([Br:9])[C:3]2=[C:7]([S:6]...,"[-1.7542141599967596, -2.764151429994894, 2.46...","[0.13901726959062444, -1.5253246683835755, 0.4...","[-0.02154374296728488, -0.021672326701921946, ...",Brc1c[nH]c2sncc12,"InChI=1/C5H3BrN2S/c6-4-2-7-5-3(4)1-8-9-5/h1-2,7H"
4,[C:1]1([H:13])=[C:5]([Br:6])[N:4]([H:14])[C:3]...,"[1.6749715699969059, -4.577603939991544, 0.892...","[-1.6464586576679636, 4.278692035598283, -0.99...","[-0.40241659410844, 0.33456349482330905, -0.06...",Brc1cc2c([nH]1)[C@@H]1C[NH2+]C[C@H]1O2,InChI=1/C8H9BrN2O/c9-7-1-5-8(11-7)4-2-10-3-6(4...
...,...,...,...,...,...,...
6799,[C:1]1([H:13])=[N:5][O:4][N:3]=[C:2]1[S:6][C:7...,"[-4.692321609991332, -2.831535599994769, -5.28...","[5.19928247168666, -2.848307882353083, 2.07801...","[0.018750378577532586, 0.21846580830943366, -0...",c1nonc1SC1CC[NH2+]CC1,InChI=1/C7H11N3OS/c1-3-8-4-2-6(1)12-7-5-9-11-1...
6800,[C:1]1([H:13])=[N:5][S:4][C:3]2=[C:2]1[C:9]([H...,"[-2.8758186299946873, -2.1326943799960603, -3....","[1.089039350778691, 2.4849390366413893, 4.4280...","[0.19253340704793356, -0.11989556449834911, -0...",c1nsc2c1C[NH2+]CC21C[NH2+]C1,InChI=1/C8H11N3S/c1-6-2-11-12-7(6)8(3-9-1)4-10...
6801,[C:1]1([H:13])=[N:5][S:4][N:3]=[C:2]1[C@:6]12[...,"[-5.011166799990743, -3.9707130999926656, -1.4...","[1.4993679527814834, 0.8681353750157257, 0.239...","[0.04265233646163001, 0.2889285700181526, -0.5...",c1nsnc1[C@@]12C[N@H+]3C[C@@H]1[C@@H]2C3,InChI=1/C8H9N3S/c1-7(10-12-9-1)8-4-11-2-5(8)6(...
6802,[C:1]1([H:10])=[C:2]2[C:3](=[C:4]([H:11])[S:5]...,"[2.0726634299961715, -6.316279109988332, -0.65...","[0.7947236276989997, 0.12701131824780987, 0.47...","[-0.3721182542703619, 0.2678683831916664, 0.26...",c1scc2c1OCCO2,"InChI=1/C6H6O2S/c1-2-8-6-4-9-3-5(6)7-1/h3-4H,1..."


In [12]:
test_array = np.array([[ 1.00447263, -1.02125479, -1.57016613],
       [ 0.11734789, -1.96305718, -1.27522016],
       [-1.12646498, -1.91658129, -0.0521976 ],
       [-0.81689407, -0.32366393,  0.59350426],
       [-1.46316775,  0.22903144,  1.54195836],
       [-0.9092707 ,  1.46898463,  1.74376757],
       [ 0.04526668,  1.61666015,  0.91381496],
       [ 0.17389752,  0.49786918,  0.12771523],
       [ 1.10515441,  0.28015903, -0.89624011],
       [ 1.90391565,  1.11645681, -1.16872337],
       [ 1.7223526 , -1.17632697, -2.35199998],
       [ 0.11406907, -2.88882606, -1.82028866],
       [ 0.67983184,  2.46821739,  0.80816263]])
find_array = test_array.flatten().tolist()

[1.00447263,
 -1.02125479,
 -1.57016613,
 0.11734789,
 -1.96305718,
 -1.27522016,
 -1.12646498,
 -1.91658129,
 -0.0521976,
 -0.81689407,
 -0.32366393,
 0.59350426,
 -1.46316775,
 0.22903144,
 1.54195836,
 -0.9092707,
 1.46898463,
 1.74376757,
 0.04526668,
 1.61666015,
 0.91381496,
 0.17389752,
 0.49786918,
 0.12771523,
 1.10515441,
 0.28015903,
 -0.89624011,
 1.90391565,
 1.11645681,
 -1.16872337,
 1.7223526,
 -1.17632697,
 -2.35199998,
 0.11406907,
 -2.88882606,
 -1.82028866,
 0.67983184,
 2.46821739,
 0.80816263]

In [ ]:
def close(conf_1: list, conf_2: list) -> bool:
    
    return np.isclose(np.array(conf_1),np.array(conf_2))


In [11]:
smiles_list = esp_store.list()
smiles_list

['C=CCC1(CO)CCCNC1',
 'CCCC#CCCC(C)O',
 'O=CCC1(CO)CC1',
 'CC1NCCS1',
 'CC(O)CCCCCC=O',
 'OCC1(CCl)OCC(F)(F)C1O',
 'OCC1(C(F)(F)F)CCNC1',
 'OCC1OCC(O)C1O',
 'CC(=O)SC1CCOC1CO',
 '[N-]=[N+]=NCC1C(O)COC1CO',
 'OCC1OCCC1Cl',
 'CC1CNCCSC1',
 'OCC1OCC(O)C1CF',
 'CCC1(CO)CCC[NH2+]C1',
 'CC(O)CCC(=O)[O-]',
 'CCC(O)c1c(C)n[nH]c1C',
 'C1CC2(CCN1)CC2',
 'O=P([O-])([O-])C1C(O)COC1CO',
 'CCC1(CO)OCCC1O',
 'CN1CCCC12CCNCC2',
 'O=S1(=O)CCNC2CCOCC21',
 'OCC1(CC2CC2)CCCN1',
 'Cc1cc(C(O)CCN)c(C)o1',
 'O=S1(=O)CCNC2CCCC21',
 'COCCC(C)O',
 'CCC(O)c1cccc(OC)c1',
 'C1CC2(CC[NH2+]CC2)CN1',
 'C=CCC1(CO)OCC(O)C1O',
 'CCCC(C)O',
 'C=CC1(O)C(O)COC1CO',
 'CC1CCS(=O)(=O)CCN1',
 'CCCC1(CO)CCC[NH2+]1',
 'OCC1OCCC1S',
 'CCC1(CO)CCCN1',
 'NCC(O)c1cccc(F)c1',
 'CC1CNCCS1(=O)=O',
 'CC(O)CC1CCCCC[NH2+]1',
 'C=CCC1(CO)SCCC1O',
 'OCC1OCCC(O)C1O',
 'C1CC2(C1)CNCCO2',
 'CC(C)(C)C1(CO)CCNC1',
 'CP(=O)([O-])OC1CCOC1CO',
 'CC(O)CC1[NH2+]CCCC1O',
 'C=CCC1(CO)CCC[NH2+]C1',
 'O=S1(=O)CCN1',
 'CC(O)CC1COC(C)(C)O1',
 'OCC1(CC2CC2)C

In [40]:
# Check for duplicates in the canon_smiles column
duplicate_counts = pandas_parquet['canon_smiles'].value_counts()
total_duplicates = duplicate_counts[duplicate_counts > 1].count()

print(f"Total number of unique canonical SMILES with duplicates: {total_duplicates}")
print("\nDuplicate counts for each canonical SMILES:")
print(duplicate_counts[duplicate_counts > 1])

# Ensure there is a match for each SMILES in smiles_list
smiles_matches = pandas_parquet['canon_smiles'].isin(smiles_list)
missing_smiles = set(smiles_list) - set(pandas_parquet['canon_smiles'])

if missing_smiles:
    print(f"\nThe following SMILES from the list are missing in the DataFrame: {missing_smiles}")
else:
    print("\nAll SMILES from the list have a match in the DataFrame.")

Total number of unique canonical SMILES with duplicates: 735

Duplicate counts for each canonical SMILES:
canon_smiles
CC(C)(CCC=O)c1ccc[nH]1           5
[H]/N=C(\NOC)Nc1nncs1            5
COC1=CC[C@](O)(CO)CC1=O          5
C=C(C=O)[C@H](O)c1ccncc1         5
CC(C)CSCCN                       5
                                ..
C=C(C)c1cnoc1C(C)(C)O            2
CC[C@@H]1C=NC[C@H](CO)[NH2+]1    2
C=C1[C@H](C)C[NH2+][C@@H]1CCO    2
N#Cc1c(CCC[NH3+])n[nH]c1N        2
CC(C)Cc1[nH]c(O)cc1O             2
Name: count, Length: 735, dtype: int64

The following SMILES from the list are missing in the DataFrame: {'[NH3+]CC1C[NH2+]CCC1(F)F', 'CCc1cc2c([nH]1)CCCC2[NH3+]', 'c1nc2c(o1)C[NH2+]CC2', 'c1nc(C2CCC2)c[nH]1', 'O=C1CCCC(=O)C1=C[O-]', 'CN1C(=O)C2CCCC21', 'CC(C)c1nnsc1C[NH3+]', 'NC1=NCCCN1', 'CC1COC(c2ccc[nH]2)=N1', 'O=Cc1nccc2cccnc12', 'CC(C)(C)C1COC[NH2+]1', 'N#CCC1C[NH2+]C(C(=O)[O-])C1', 'C1CCC(c2nn[n-]n2)C1', 'CC(O)C([NH3+])O', 'CC(C)[NH2+]C(C)C#N', 'CC(C)(C)Cc1ncc(C=O)s1', 'O=C([O-])C1CCc

In [16]:
[i for i in smiles_list if i == is_in]

['BrC1(Br)CC1']

In [17]:
smiles_list.count(is_in)

1

In [5]:
table = pq.read_table('testing.parquet')
pandas_parquet = table.to_pandas()
pandas_parquet['esp_column'] = np.nan
pandas_parquet['inv_distance_column'] = np.nan

def inv_distance(conformer: list, grid: unit.Quantity):
    
    conformer = np.array(conformer).reshape(-1,3) * unit.bohr
    grid = grid.to(unit.bohr)
    displacement = grid[:, None, :] - conformer[None, :, :]  # N x M x 3 B
    distance = np.linalg.norm(displacement.m, axis=-1) * unit.bohr # N, M
    inv_distance = 1 / distance  #N, M
    return inv_distance.m
    
for smile in smiles_list:
    record = esp_store.retrieve(smiles = smile)
    tagged_smiles = record[0].tagged_smiles
    conformer = record[0].conformer
    grid = record[0].grid_coordinates_quantity
    if tagged_smiles in pandas_parquet['smiles'].values:
        pandas_parquet.loc[pandas_parquet['smiles'] == tagged_smiles, 'esp_column'] = record[0].esp
        conformer_data = pandas_parquet.loc[pandas_parquet['smiles'] == tagged_smiles, 'conformation'].values[0]
        inv_dist_data = inv_distance(conformer = conformer_data, grid = grid)
        pandas_parquet.loc[pandas_parquet['smiles'] == tagged_smiles, 'inv_distance_column'] = inv_dist_data.flatten()

updated_table = pa.Table.from_pandas(pandas_parquet)
pq.write_table(pandas_parquet, "testing_esp.parquet")

In [ ]:
esp_store.retrieve(smiles= 'O=c1[nH]c(C2CCCC2)cs1',)[0].tagged_smiles

'[C:1]1([H:12])=[C:2]([C:7]2([H:14])[C:8]([H:15])([H:16])[C:9]([H:17])([H:18])[C:10]([H:19])([H:20])[C:11]2([H:21])[H:22])[N:3]([H:13])[C:4](=[O:5])[S:6]1'

In [3]:
esp_store.retrieve(smiles='O=c1ccsc2nncn12')[0].tagged_smiles

'[C:1]1([H:11])=[C:2]([H:12])[S:3][C:4]2=[N:5][N:6]=[C:7]([H:13])[N:8]2[C:9]1=[O:10]'

In [13]:
esp_store.retrieve(smiles= 'Cn1cc(C(C)(C)[NH3+])cn1')[0].tagged_smiles

'[C:1]([C:2]([C:3]([H:14])([H:15])[H:16])([C:4]1=[C:8]([H:18])[N:7]([C:9]([H:19])([H:20])[H:21])[N:6]=[C:5]1[H:17])[N+:10]([H:22])([H:23])[H:24])([H:11])([H:12])[H:13]'

In [71]:
conf = esp_store.retrieve(smiles='O=c1ccsc2nncn12')[0].conformer

In [70]:
conf_quantity = esp_store.retrieve(smiles='O=c1ccsc2nncn12')[0].conformer_quantity

In [82]:
input_smiles = 'O=c1ccsc2nncn12'
inchi_result = inchi(input_smiles, conf.flatten().tolist())   
inchi_result

'InChI=1/C5H3N3OS/c9-4-1-2-10-5-7-6-3-8(4)5/h1-3H'

In [83]:
inchi_result in pandas_parquet['inch']

False

In [78]:
pandas_parquet['inch']

0                     InChI=1/C3H4Br2/c4-3(5)1-2-3/h1-2H2
1                              InChI=1/CH2Br2/c2-1-3/h1H2
2       InChI=1/C7H5BrN2/c8-7-5-3-1-2-4-6(5)9-10-7/h1-...
3        InChI=1/C5H3BrN2S/c6-4-2-7-5-3(4)1-8-9-5/h1-2,7H
4       InChI=1/C8H9BrN2O/c9-7-1-5-8(11-7)4-2-10-3-6(4...
                              ...                        
6799    InChI=1/C7H11N3OS/c1-3-8-4-2-6(1)12-7-5-9-11-1...
6800    InChI=1/C8H11N3S/c1-6-2-11-12-7(6)8(3-9-1)4-10...
6801    InChI=1/C8H9N3S/c1-7(10-12-9-1)8-4-11-2-5(8)6(...
6802    InChI=1/C6H6O2S/c1-2-8-6-4-9-3-5(6)7-1/h3-4H,1...
6803    InChI=1/C4H2N6O2/c5-1-2(8-11-7-1)6-4-3(5)9-12-...
Name: inch, Length: 6804, dtype: object

In [14]:
esp_store.retrieve(smiles='O=c1ccsc2nncn12')[0].esp_quantity

Magnitude,[[0.0016401063367581514] [0.0030256625926892156] [0.001373838385418935] ... [-0.012885789431193118] [-0.011292468272296574] [-0.012301091918553553]]
Units,hartree/elementary_charge


In [16]:
esp_store.retrieve(smiles='O=c1ccsc2nncn12')[0].esp

array([[ 0.00164011],
       [ 0.00302566],
       [ 0.00137384],
       ...,
       [-0.01288579],
       [-0.01129247],
       [-0.01230109]])

In [12]:
retrieved =  esp_store.retrieve(smiles='[N-]=[N+]=NCC1C(O)COC1CO')
retrieved[0].grid_coordinates_quantity

Magnitude,[[-6.8260902074322125 -3.0999495106630994 -0.2663685482193558] [-6.8260902074322125 -3.0999495106630994 0.44073823296719167] [-6.8260902074322125 -2.7463961200698255 -0.6199219388126296] ... [4.487618291552547 1.1426911764561853 1.147845014153739] [4.487618291552547 1.496244567049459 0.08718484237391794] [4.487618291552547 1.496244567049459 0.7942916235604653]]
Units,angstrom


In [5]:
esp_store.retrieve(smiles='O=c1ccsc2nncn12', method='hf')[0].grid_coordinates_quantity

Magnitude,[[-4.400000940060713 -3.306006021563344 0.09156008226150186] [-4.400000940060713 -2.95245263097007 -0.9691000895183193] [-4.400000940060713 -2.95245263097007 -0.26199330833177187] ... [4.792387215364404 1.6437414467424887 -1.322653480111593] [4.792387215364404 1.6437414467424887 -0.6155466989250455] [4.792387215364404 1.9972948373357624 -0.9691000895183193]]
Units,angstrom
